<a href="https://colab.research.google.com/github/melikas/ABC/blob/main/PyActigraphy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  statsmodels
!pip install --upgrade pandas
!pip install --upgrade numpy
!pip install --upgrade matplotlib
!pip install kaleido
!pip install fastapi
!pip install python-multipart
!pip install uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 9.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires uvicorn, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00


In [ ]:
!pip install pyActigraphy

In [ ]:
!pip install --upgrade scipy


  Using cached scipy-1.11.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (36.4 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerometer 7.1.1 requires scipy==1.7.*, but you have scipy 1.11.4 which is incompatible.
jax 0.4.23 requires numpy>=1.22, but you have numpy 1.21.6 which is incompatible.
jaxlib 0.4.23+cuda12.cudnn89 requires numpy>=1.22, but you have numpy 1.21.6 which is incompatible.
plotnine 0.12.4 requires matplotlib>=3.6.0, but you have matplotlib 3.5.3 which is incompatible.
plotnine 0.12.4 requires numpy>=1.23.0, but you have numpy 1.21.6 which is incompatible.
plotnine 0.12.4 requires pandas>=1.5.0, but you have pandas 1.3.5 which is incompatible.
plotnine 0.12.4 requires statsm

In [ ]:
!pip install pandas

In [ ]:
! pip install numpy==1.25.0

In [ ]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
import pyActigraphy

In [ ]:
# Define the path to your uploaded zip file
zip_path = '/content/depresjon.zip'  # Make sure to replace 'your_zip_file.zip' with your actual file name

# Define the directory where you want to extract the files
extracted_path = '/content/extracted/'

# Create the directory if it doesn't exist
os.makedirs(extracted_path, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

# List the files in the extracted directory
extracted_files = os.listdir(extracted_path)

# Print the list of files
print("Extracted Files:")
for file in extracted_files:
    print(file)

Extracted Files:
data


In [ ]:
from pandas.tseries import frequencies
extracted_data = pd.read_csv('/content/extracted/data/control/control_1.csv')
d = {'timestamp': extracted_data['timestamp'], 'date': extracted_data['date'], 'activity': extracted_data['activity']}
index = pd.date_range(start='18/03/2003  3:00:00 PM',freq='60s',periods=51611)
data = pd.DataFrame(data=d, index=index)
data['timestamp'] = pd.to_datetime(data['timestamp'])
#data.set_index('timestamp', inplace=True)
data['activity'] = extracted_data['activity'].values
data['timestamp'] = extracted_data['timestamp'].values
data['date'] = extracted_data['date'].values


In [ ]:
data

,timestamp,date,activity
2003-03-18 15:00:00,2003-03-18 15:00:00,2003-03-18,60
2003-03-18 15:01:00,2003-03-18 15:01:00,2003-03-18,0
2003-03-18 15:02:00,2003-03-18 15:02:00,2003-03-18,264
2003-03-18 15:03:00,2003-03-18 15:03:00,2003-03-18,662
2003-03-18 15:04:00,2003-03-18 15:04:00,2003-03-18,293
...,...,...,...
2003-04-23 11:06:00,2003-04-23 12:06:00,2003-04-23,3
2003-04-23 11:07:00,2003-04-23 12:07:00,2003-04-23,3
2003-04-23 11:08:00,2003-04-23 12:08:00,2003-04-23,3
2003-04-23 11:09:00,2003-04-23 12:09:00,2003-04-23,3


In [ ]:
from pyActigraphy.io import BaseRaw
raw = BaseRaw(
    name="control_1",
    uuid='DeviceId',
    format='Pandas',
    axial_mode=None,
    start_time=data.index[0],
    period=(data.index[-1]-data.index[0]),
    frequency=index.freq,
    data=data['activity'],
    light=None
)

In [ ]:
raw.data

2003-03-18 15:00:00     60
2003-03-18 15:01:00      0
2003-03-18 15:02:00    264
2003-03-18 15:03:00    662
2003-03-18 15:04:00    293
                      ... 
2003-04-23 11:06:00      3
2003-04-23 11:07:00      3
2003-04-23 11:08:00      3
2003-04-23 11:09:00      3
2003-04-23 11:10:00      0
Freq: 60S, Name: activity, Length: 51611, dtype: int64

In [ ]:
import plotly.graph_objects as go
from pyActigraphy.analysis import Cosinor
cosinor = Cosinor()
cosinor.fit_initial_params['Period'].vary = False
cosinor.fit_initial_params.pretty_print()


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
Acrophase     3.142        0    6.283     None     True     None     None
Amplitude        50        0      inf     None     True     None     None
Mesor            50        0      inf     None     True     None     None
Period         1440        0      inf     None    False     None     None


In [ ]:
results = cosinor.fit(raw.data, verbose=True) # Set verbose to True to print the fit output

[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 26
    # data points      = 51611
    # variables        = 3
    chi-square         = 4.1802e+09
    reduced chi-square = 80998.9492
    Akaike info crit   = 583320.403
    Bayesian info crit = 583346.958
[[Variables]]
    Amplitude:  76.2461944 +/- 1.77241552 (2.32%) (init = 50)
    Acrophase:  6.10828460 +/- 0.02320013 (0.38%) (init = 3.141593)
    Period:     1440 (fixed)
    Mesor:      98.7980042 +/- 1.25278472 (1.27%) (init = 50)


In [ ]:
results.params.valuesdict()

{'Amplitude': 76.24619436726096,
 'Acrophase': 6.108284601763934,
 'Period': 1440,
 'Mesor': 98.79800416854177}

In [ ]:
path = '/content/extracted/data/control/'
dirs = os.listdir(path)
cosinor_result = []
from pyActigraphy.io import BaseRaw
import plotly.graph_objects as go
from pyActigraphy.analysis import Cosinor

print(dirs)
for filepath in dirs:
    source = filepath.split('.')[0]
    if filepath.endswith('.csv'):
      extracted_data = pd.read_csv(path + filepath)
      d = {'timestamp': extracted_data['timestamp'], 'date': extracted_data['date'], 'activity': extracted_data['activity']}
      index = pd.date_range(start=extracted_data['timestamp'].min(),freq='60s',periods=len(extracted_data))
      data = pd.DataFrame(data=d, index=index)
      data['timestamp'] = pd.to_datetime(data['timestamp'])
      #data.set_index('timestamp', inplace=True)
      data['activity'] = extracted_data['activity'].values
      data['timestamp'] = extracted_data['timestamp'].values
      data['date'] = extracted_data['date'].values

      raw = BaseRaw(
            name=source,
            uuid='DeviceId',
            format='Pandas',
            axial_mode=None,
            start_time=data.index[0],
            period=(data.index[-1]-data.index[0]),
            frequency=index.freq,
            data=data['activity'],
            light=None
      )
      cosinor = Cosinor()
      cosinor.fit_initial_params['Period'].vary = False
      results = cosinor.fit(raw.data, verbose=False) # Set verbose to True to print the fit output
      loop_result = results.params.valuesdict()
      loop_result['File_Name'] = raw.name
      cosinor_result.append(loop_result)
df_controls = pd.DataFrame(cosinor_result)
print(df_controls)


['control_7.csv', 'control_3.csv', 'control_10.csv', 'control_12.csv', 'control_31.csv', 'control_9.csv', 'control_32.csv', 'control_19.csv', 'control_22.csv', 'control_28.csv', 'control_24.csv', 'control_13.csv', 'control_1.csv', 'control_27.csv', 'control_18.csv', 'control_23.csv', 'control_4.csv', 'control_25.csv', 'control_8.csv', 'control_20.csv', 'control_29.csv', 'control_15.csv', 'control_26.csv', 'control_11.csv', 'control_16.csv', 'control_14.csv', 'control_6.csv', 'control_30.csv', 'control_17.csv', 'control_5.csv', 'control_21.csv', 'control_2.csv']
       Amplitude  Acrophase  Period       Mesor   File_Name
0   9.990838e+01   5.903885    1440  163.176973   control_7
1   2.918315e-09   3.173090    1440   76.906412   control_3
2   2.171203e+02   5.087482    1440  280.148785  control_10
3   8.432946e+01   4.930164    1440   93.235480  control_12
4   1.227430e+02   5.858825    1440  147.632114  control_31
5   9.146469e+01   4.489120    1440  107.059191   control_9
6   3.959171

In [ ]:
path = '/content/extracted/data/condition/'
dirs = os.listdir(path)
cosinor_result = []
from pyActigraphy.io import BaseRaw
import plotly.graph_objects as go
from pyActigraphy.analysis import Cosinor

print(dirs)
for filepath in dirs:
    source = filepath.split('.')[0]
    if filepath.endswith('.csv'):
      extracted_data = pd.read_csv(path + filepath)
      d = {'timestamp': extracted_data['timestamp'], 'date': extracted_data['date'], 'activity': extracted_data['activity']}
      index = pd.date_range(start=extracted_data['timestamp'].min(),freq='60s',periods=len(extracted_data))
      data = pd.DataFrame(data=d, index=index)
      data['timestamp'] = pd.to_datetime(data['timestamp'])
      #data.set_index('timestamp', inplace=True)
      data['activity'] = extracted_data['activity'].values
      data['timestamp'] = extracted_data['timestamp'].values
      data['date'] = extracted_data['date'].values

      raw = BaseRaw(
            name=source,
            uuid='DeviceId',
            format='Pandas',
            axial_mode=None,
            start_time=data.index[0],
            period=(data.index[-1]-data.index[0]),
            frequency=index.freq,
            data=data['activity'],
            light=None
      )
      cosinor = Cosinor()
      cosinor.fit_initial_params['Period'].vary = False
      results = cosinor.fit(raw.data, verbose=False) # Set verbose to True to print the fit output
      loop_result = results.params.valuesdict()
      loop_result['File_Name'] = raw.name
      cosinor_result.append(loop_result)
df_conditions = pd.DataFrame(cosinor_result)
print(df_conditions)

['condition_6.csv', 'condition_10.csv', 'condition_22.csv', 'condition_12.csv', 'condition_4.csv', 'condition_17.csv', 'condition_5.csv', 'condition_21.csv', 'condition_3.csv', 'condition_8.csv', 'condition_9.csv', 'condition_23.csv', 'condition_2.csv', 'condition_11.csv', 'condition_15.csv', 'condition_18.csv', 'condition_7.csv', 'condition_1.csv', 'condition_14.csv', 'condition_19.csv', 'condition_16.csv', 'condition_20.csv', 'condition_13.csv']
       Amplitude  Acrophase  Period       Mesor     File_Name
0   1.674912e+02   5.424896    1440  196.849262   condition_6
1   2.671808e+02   4.624900    1440  289.543232  condition_10
2   1.263917e+02   5.162857    1440  161.040680  condition_22
3   1.388930e+02   4.605599    1440  149.082491  condition_12
4   1.979003e+02   5.112550    1440  274.986956   condition_4
5   6.201826e+01   4.544390    1440   85.779763  condition_17
6   1.591926e+02   4.939013    1440  167.238922   condition_5
7   7.532503e+01   6.025280    1440   78.488580  con

In [ ]:
result_concat_rows = pd.concat([df_controls, df_conditions], axis=0)
print(result_concat_rows)

       Amplitude  Acrophase  Period       Mesor     File_Name
0   9.990838e+01   5.903885    1440  163.176973     control_7
1   2.918315e-09   3.173090    1440   76.906412     control_3
2   2.171203e+02   5.087482    1440  280.148785    control_10
3   8.432946e+01   4.930164    1440   93.235480    control_12
4   1.227430e+02   5.858825    1440  147.632114    control_31
5   9.146469e+01   4.489120    1440  107.059191     control_9
6   3.959171e+01   5.586085    1440   68.313494    control_32
7   1.088285e+02   4.597274    1440  169.044452    control_19
8   1.788552e+02   4.551430    1440  245.194708    control_22
9   1.760811e+02   4.832174    1440  293.338486    control_28
10  3.031218e+02   4.089611    1440  352.320614    control_24
11  1.522290e+02   4.946746    1440  165.183504    control_13
12  7.624619e+01   6.108285    1440   98.798004     control_1
13  1.970146e+02   4.390530    1440  312.926541    control_27
14  1.244799e-06   1.846523    1440  241.480898    control_18
15  1.26

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Assuming your DataFrame is named df
# 'label' is the column indicating whether it's a control (0) or condition (1) instance
# Drop 'File_Name' and 'label' from X as they are not features for the model
# Create a binary label column (1 for 'condition', 0 for 'control')
result_concat_rows['LABEL'] = result_concat_rows['File_Name'].str.contains('condition').astype(int)

# Features are all columns except 'FILE_NAME' and the label column
X = result_concat_rows.drop(['File_Name', 'LABEL'], axis=1)

# Target variable is the 'LABEL' column
y = result_concat_rows['LABEL']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Logistic Regression
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

# Predictions
linear_predictions = linear_model.predict(X_test)
logistic_predictions = logistic_model.predict(X_test)

# Evaluation
linear_accuracy = accuracy_score(y_test, linear_predictions.round())
logistic_accuracy = accuracy_score(y_test, logistic_predictions)

print(f'Linear Regression Accuracy: {linear_accuracy}')
print(f'Logistic Regression Accuracy: {logistic_accuracy}')

# Additional metrics for logistic regression
logistic_classification_report = classification_report(y_test, logistic_predictions)
print('Logistic Regression Classification Report:\n', logistic_classification_report)


Linear Regression Accuracy: 0.6363636363636364
Logistic Regression Accuracy: 0.7272727272727273
Logistic Regression Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.67      0.73         6
           1       0.67      0.80      0.73         5

    accuracy                           0.73        11
   macro avg       0.73      0.73      0.73        11
weighted avg       0.74      0.73      0.73        11



In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create a Random Forest classifier
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

# Predictions
rf_predictions = rf_model.predict(X_test)

# Evaluation
rf_accuracy = accuracy_score(y_test, rf_predictions)
print(f'Random Forest Accuracy: {rf_accuracy}')


Random Forest Accuracy: 0.45454545454545453


In [ ]:
from xgboost import XGBClassifier

# Create an XGBoost classifier
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

# Predictions
xgb_predictions = xgb_model.predict(X_test)

# Evaluation
xgb_accuracy = accuracy_score(y_test, xgb_predictions)
print(f'XGBoost Accuracy: {xgb_accuracy}')


XGBoost Accuracy: 0.6363636363636364


In [ ]:
from sklearn.svm import SVC

# Create a Support Vector Machine classifier
svm_model = SVC()
svm_model.fit(X_train, y_train)

# Predictions
svm_predictions = svm_model.predict(X_test)

# Evaluation
svm_accuracy = accuracy_score(y_test, svm_predictions)
print(f'SVM Accuracy: {svm_accuracy}')


SVM Accuracy: 0.5454545454545454


In [ ]:
%%writefile depresjon_cosinor.py

!pip install pyActigraphy

import os
from datetime import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
import pyActigraphy
from pyActigraphy.io import BaseRaw



# Define the path to your uploaded zip file
zip_path = '/content/depresjon.zip'  # Make sure to replace 'your_zip_file.zip' with your actual file name

# Define the directory where you want to extract the files
extracted_path = '/content/extracted/'

# Create the directory if it doesn't exist
os.makedirs(extracted_path, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

# List the files in the extracted directory
extracted_files = os.listdir(extracted_path)

# Print the list of files
print("Extracted Files:")
for file in extracted_files:
    print(file)


path = '/content/extracted/data/control/'
dirs = os.listdir(path)
cosinor_result = []
from pyActigraphy.io import BaseRaw
import plotly.graph_objects as go
from pyActigraphy.analysis import Cosinor

print(dirs)
for filepath in dirs:
    source = filepath.split('.')[0]
    if filepath.endswith('.csv'):
      extracted_data = pd.read_csv(path + filepath)
      d = {'timestamp': extracted_data['timestamp'], 'date': extracted_data['date'], 'activity': extracted_data['activity']}
      index = pd.date_range(start=extracted_data['timestamp'].min(),freq='60s',periods=len(extracted_data))
      data = pd.DataFrame(data=d, index=index)
      data['timestamp'] = pd.to_datetime(data['timestamp'])
      #data.set_index('timestamp', inplace=True)
      data['activity'] = extracted_data['activity'].values
      data['timestamp'] = extracted_data['timestamp'].values
      data['date'] = extracted_data['date'].values

      raw = BaseRaw(
            name=source,
            uuid='DeviceId',
            format='Pandas',
            axial_mode=None,
            start_time=data.index[0],
            period=(data.index[-1]-data.index[0]),
            frequency=index.freq,
            data=data['activity'],
            light=None
      )
      cosinor = Cosinor()
      cosinor.fit_initial_params['Period'].vary = False
      results = cosinor.fit(raw.data, verbose=False) # Set verbose to True to print the fit output
      loop_result = results.params.valuesdict()
      loop_result['File_Name'] = raw.name
      cosinor_result.append(loop_result)
df_controls = pd.DataFrame(cosinor_result)
print(df_cosinor_result)




path = '/content/extracted/data/condition/'
dirs = os.listdir(path)
cosinor_result = []
from pyActigraphy.io import BaseRaw
import plotly.graph_objects as go
from pyActigraphy.analysis import Cosinor

print(dirs)
for filepath in dirs:
    source = filepath.split('.')[0]
    if filepath.endswith('.csv'):
      extracted_data = pd.read_csv(path + filepath)
      d = {'timestamp': extracted_data['timestamp'], 'date': extracted_data['date'], 'activity': extracted_data['activity']}
      index = pd.date_range(start=extracted_data['timestamp'].min(),freq='60s',periods=len(extracted_data))
      data = pd.DataFrame(data=d, index=index)
      data['timestamp'] = pd.to_datetime(data['timestamp'])
      #data.set_index('timestamp', inplace=True)
      data['activity'] = extracted_data['activity'].values
      data['timestamp'] = extracted_data['timestamp'].values
      data['date'] = extracted_data['date'].values

      raw = BaseRaw(
            name=source,
            uuid='DeviceId',
            format='Pandas',
            axial_mode=None,
            start_time=data.index[0],
            period=(data.index[-1]-data.index[0]),
            frequency=index.freq,
            data=data['activity'],
            light=None
      )
      cosinor = Cosinor()
      cosinor.fit_initial_params['Period'].vary = False
      results = cosinor.fit(raw.data, verbose=False) # Set verbose to True to print the fit output
      loop_result = results.params.valuesdict()
      loop_result['File_Name'] = raw.name
      cosinor_result.append(loop_result)
df_conditions = pd.DataFrame(cosinor_result)
print(df_conditions)



result_concat_rows = pd.concat([df_controls, df_conditions], axis=0)
print(result_concat_rows)


import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Assuming your DataFrame is named df
# 'label' is the column indicating whether it's a control (0) or condition (1) instance
# Drop 'File_Name' and 'label' from X as they are not features for the model
# Create a binary label column (1 for 'condition', 0 for 'control')
result_concat_rows['LABEL'] = result_concat_rows['File_Name'].str.contains('condition').astype(int)

# Features are all columns except 'FILE_NAME' and the label column
X = result_concat_rows.drop(['File_Name', 'LABEL'], axis=1)

# Target variable is the 'LABEL' column
y = result_concat_rows['LABEL']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Logistic Regression
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

# Predictions
linear_predictions = linear_model.predict(X_test)
logistic_predictions = logistic_model.predict(X_test)

# Evaluation
linear_accuracy = accuracy_score(y_test, linear_predictions.round())
logistic_accuracy = accuracy_score(y_test, logistic_predictions)

print(f'Linear Regression Accuracy: {linear_accuracy}')
print(f'Logistic Regression Accuracy: {logistic_accuracy}')

# Additional metrics for logistic regression
logistic_classification_report = classification_report(y_test, logistic_predictions)
print('Logistic Regression Classification Report:\n', logistic_classification_report)


Writing depresjon_cosinor.py
